In [ ]:
% matplotlib inline
import bidict
import os
import csv
import scipy.sparse
import numpy as np
import random
import matplotlib.pyplot as plt
import load_data
from blessings import Terminal
term = Terminal()
print term.blue('kotek')

In [ ]:
# constants
all_compounds_file = 'Random_compounds_100.sdf'
folder_with_pairs = 'pairs'
n_sets = 10

possible_compounds = 'possible compounds'
pairs_contained = 'pairs contained'
probability = 'probability'
compounds_contained = 'compounds contained'
size = 'size'

In [ ]:
def clustering_score(clusters, n_omitted):
    # chcemy niską wariancję i mało pominiętych par
    assert isinstance(clusters, list), 'clusters should be a non-empty list of dictionaries'
    assert len(clusters) > 0, 'clusters should be a non-empty list of dictionaries'
    assert isinstance(clusters[0], dict), 'clusters should be a non-empty list of dictionaries'
    variance = 0
    if pairs_contained in clusters[0].keys():
        variance = np.var([float(len(cluster[pairs_contained]))/(cluster[probability][1]-cluster[probability][0])
                       for cluster in clusters])
    elif size in clusters[0].keys():
        variance = np.var([float(cluster[size])/(cluster[probability][1]-cluster[probability][0])
                       for cluster in clusters])
    else:
        raise KeyError('either `'+size+'` or `'+pairs_contained+'` must be in clusters')
    score = (5./8)*variance + n_omitted
    return score, variance # TODO 30 - powinno byc wiecej, bo klastrow bedzie mniej

In [ ]:
def agnieszka_folds(all_compunds_file, folder_with_pairs):
    best_clusterings = []
    best_score = 10**4
    scores = []
    variances = []
    n_ommited_pairs = []
    while(True): # many times
        bin_sim, _, mapping_idx_chembl = load_data.load_similarity_matrices(all_compunds_file, folder_with_pairs)
        bin_sim.setdiag(np.zeros((bin_sim.shape[0]))) # pairs (i,i) are not interesting for us
        pairs = zip(bin_sim.nonzero()[0], bin_sim.nonzero()[1])
                
        random.shuffle(pairs)
        clusters = [{possible_compounds:list(mapping_idx_chembl.keys()), pairs_contained:[]} for i in xrange(2)]
        clusters[0][probability], clusters[1][probability] = (0.0, 0.9), (0.9, 1.0)
        failures = 0
        max_failures = len(pairs)
        try:
            while failures < max_failures:
                i, j = pairs.pop() # popping random pair
                if i > j:
                    continue # saving time
                
                x = np.random.rand()
                for cluster in clusters:  # iterating over clusters to find the chosen one
                    if cluster[probability][0] <= x < cluster[probability][1]:  # if the cluster was chosen
                        # if pair might go inside
                        if i in cluster[possible_compounds] and j in cluster[possible_compounds]:
                            failures = 0
                            cluster[pairs_contained].append((i,j))
                            
                            # usun zwiazki i, j z mozliwych w kazdym klastrze poza obecnym
                            for c in clusters:
                                if i in c[possible_compounds]:
                                    c[possible_compounds].remove(i)
                                if j in c[possible_compounds]:
                                    c[possible_compounds].remove(j)
                            cluster[possible_compounds].extend([i, j])
                        else: # the cluster was chosen but we cannot fit the pair there
                            failures += 1
                            pairs.insert(0, (i, j)) # pair goes back to the poll
                        break # we've found the chosen one
                
        except IndexError:  # zbior par jest pusty
            print term.green('index error')
            pass
        finally:
            # wyszlismy z while'a, wiec albo mamy wpasowane wszystkie pary albo mielismy 100 porazek
            omitted_pairs = len(pairs)
            score, variance = clustering_score(clusters, omitted_pairs)
            
            # TODO: new score needed
            print 'score', score
            print 'omitted_pairs', omitted_pairs
            if score < best_score:
                print '\n'
                
                best_score = score
                best_clusterings.append(clusters)
                if len(best_clusterings) > n_sets:
                    best_clusterings = best_clusterings[-n_sets:]
                scores.append(best_score)
                variances.append(variance)
                n_ommited_pairs.append(omitted_pairs)
                plt.plot(range(len(scores)), scores, c='r')
                plt.plot(range(len(variances)), variances, c='g')
                plt.plot(range(len(n_ommited_pairs)), n_ommited_pairs, c='b')
                plt.title('best score over time')
                plt.show()
                print "score", score, 'variance', variance, 'omitted_pairs', omitted_pairs
                print 'wielkości klastrów', [len(cluster[pairs_contained]) for cluster in clusters]
                print 'pominietych par', omitted_pairs
                print "#BEST CLUSTERING\n", best_clusterings[-1]

    
            print '_________________________________________________\n'
    
    return best_clusterings

In [ ]:
best_clustering = agnieszka_folds(all_compounds_file, folder_with_pairs)

In [ ]:
def staszek_folds(all_compounds_file, folder_with_pairs):
    best_clusterings = []
    best_score = 10**4
    scores = []
    variances = []
    n_ommited_pairs = []
    
    while(True): # many times
        bin_sim, _, mapping_idx_chembl = load_data.load_similarity_matrices(all_compounds_file, folder_with_pairs)
        bin_sim.setdiag(np.zeros((bin_sim.shape[0]))) # pairs (i,i) are not interesting for us
        pairs = zip(bin_sim.nonzero()[0], bin_sim.nonzero()[1])
                
        random.shuffle(pairs)
        clusters = [{compounds_contained:[], size:0} for i in xrange(2)]
        clusters[0][probability], clusters[1][probability] = (0.0, 0.9), (0.9, 1.0)
        
        omitted_pairs = 0
        
        for compound_index in xrange(0, len(pairs)):
            x = np.random.rand()
            for cluster in clusters:  # iterating over clusters to find the chosen one
                    if cluster[probability][0] <= x < cluster[probability][1]:  # if the cluster was chosen
                        cluster[compounds_contained].append(compound_index)
                        
        for pair in pairs:
            i, j = pair
            if i > j:
                continue # saving time
                
            contained = False
            for cluster in clusters:
                if i in cluster[compounds_contained] and j in cluster[compounds_contained]:
                    cluster[size] += 1
                    contained=True
                    break
                    
            if not contained:    
                omitted_pairs += 1

        # scoring the clustering found
        score, variance = clustering_score(clusters, omitted_pairs)
        print 'omitted_pairs', omitted_pairs
        print 'score', score
        if score < best_score:
            print '\n'

            best_score = score
            best_clusterings.append(clusters)
            if len(best_clusterings) > n_sets:
                    best_clusterings = best_clusterings[-n_sets:]
            scores.append(best_score)
            variances.append(variance)
            n_ommited_pairs.append(omitted_pairs)
            plt.plot(range(len(scores)), scores, c='r')
            plt.plot(range(len(variances)), variances, c='g')
            plt.plot(range(len(n_ommited_pairs)), n_ommited_pairs, c='b')
            plt.title('best score over time')
            plt.show()
            print "score", score, 'variance', variance, 'omitted_pairs', omitted_pairs
            print 'wielkości klastrów', [cluster[size] for cluster in clusters]
            print 'pominietych par', omitted_pairs
            print "#BEST CLUSTERING\n", best_clusterings[-1]

    
        print '_________________________________________________\n'
    
    return best_clusterings

In [ ]:
best_clustering = staszek_folds(all_compounds_file, folder_with_pairs)